In [1]:
import numpy as np
from pathlib import Path
import cv2

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.autograd import Variable
from torchvision.utils import save_image

In [2]:
import sys
import os
# sys.path.append('../src/oomugi/')
sys.path.append(os.path.abspath('../src/oomugi/'))
import pprint
pprint.pprint(sys.path)

from model import UNet
from dataset import SSSDataset
from loss import DiscriminativeLoss

['',
 'C:\\Users\\y_kato\\Anaconda3\\envs\\pytorch0.3.1\\python36.zip',
 'C:\\Users\\y_kato\\Anaconda3\\envs\\pytorch0.3.1\\DLLs',
 'C:\\Users\\y_kato\\Anaconda3\\envs\\pytorch0.3.1\\lib',
 'C:\\Users\\y_kato\\Anaconda3\\envs\\pytorch0.3.1',
 'C:\\Users\\y_kato\\Anaconda3\\envs\\pytorch0.3.1\\lib\\site-packages',
 'C:\\Users\\y_kato\\Anaconda3\\envs\\pytorch0.3.1\\lib\\site-packages\\win32',
 'C:\\Users\\y_kato\\Anaconda3\\envs\\pytorch0.3.1\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\y_kato\\Anaconda3\\envs\\pytorch0.3.1\\lib\\site-packages\\Pythonwin',
 'C:\\Users\\y_kato\\Anaconda3\\envs\\pytorch0.3.1\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\y_kato\\.ipython',
 'I:\\ykato_git\\omg_instance_segmentation\\src\\pytorch-discriminative-loss-master\\src\\oomugi']


In [3]:
n_sticks = 8

In [4]:
# Model
model = UNet().cuda()
model.load_state_dict(torch.load('../model/model.pth'))

In [5]:
# Dataset for train
train_dataset = SSSDataset(train=True, n_sticks=n_sticks)
train_dataloader = DataLoader(train_dataset, batch_size=1,
                              shuffle=False, num_workers=0, pin_memory=True)

In [6]:
# Loss Function
criterion_disc = DiscriminativeLoss(delta_var=0.5,
                                    delta_dist=1.5,
                                    norm=2,
                                    usegpu=True).cuda()
criterion_ce = nn.CrossEntropyLoss().cuda()

In [7]:
# Optimizer
parameters = model.parameters()
optimizer = optim.SGD(parameters, lr=0.01, momentum=0.9, weight_decay=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,
                                                 mode='min',
                                                 factor=0.1,
                                                 patience=10,
                                                 verbose=True)

In [8]:
for i in range(100 + 1):
    img_path = 'I:/ykato_git/datasets/oomugi_blender/test/o/{:04}.png'.format(i)
    img = cv2.imread(img_path)
    print(img_path, type(img))
    img = cv2.resize(img, (128, 128), interpolation= cv2.INTER_NEAREST)
    img = img.transpose(2,0,1)
    img = torch.Tensor(img[np.newaxis].astype(np.uint8))
    img = Variable(img).cuda()
    sem_predict, ins_predict = model(img)
    print(sem_predict.cpu().data.numpy().shape)
    
    save_dir = './predict/{}'.format(i)
    try:
        os.mkdir(save_dir)
    except:
        pass
    print(sem_predict.cpu().data.numpy()[0,0].shape)
    sem = sem_predict.cpu().data.numpy()[0,0] * 255
#     cv2.imwrite(os.path.join(save_dir, '{:04}.png'.format(i)), sem)
    cv2.imwrite('./predict/{:04}.png'.format(i), sem)
    
    for j in range(ins_predict.shape[1]):
        ins = ins_predict.cpu().data.numpy()[0,j] * 255
        cv2.imwrite(os.path.join(save_dir, 'ins{}.png'.format(j)), ins)
    
    

I:/ykato_git/datasets/oomugi_blender/test/o/0000.png <class 'numpy.ndarray'>
(1, 2, 128, 128)
(128, 128)
I:/ykato_git/datasets/oomugi_blender/test/o/0001.png <class 'numpy.ndarray'>
(1, 2, 128, 128)
(128, 128)
I:/ykato_git/datasets/oomugi_blender/test/o/0002.png <class 'numpy.ndarray'>
(1, 2, 128, 128)
(128, 128)
I:/ykato_git/datasets/oomugi_blender/test/o/0003.png <class 'numpy.ndarray'>
(1, 2, 128, 128)
(128, 128)
I:/ykato_git/datasets/oomugi_blender/test/o/0004.png <class 'numpy.ndarray'>
(1, 2, 128, 128)
(128, 128)
I:/ykato_git/datasets/oomugi_blender/test/o/0005.png <class 'numpy.ndarray'>
(1, 2, 128, 128)
(128, 128)
I:/ykato_git/datasets/oomugi_blender/test/o/0006.png <class 'numpy.ndarray'>
(1, 2, 128, 128)
(128, 128)
I:/ykato_git/datasets/oomugi_blender/test/o/0007.png <class 'numpy.ndarray'>
(1, 2, 128, 128)
(128, 128)
I:/ykato_git/datasets/oomugi_blender/test/o/0008.png <class 'numpy.ndarray'>
(1, 2, 128, 128)
(128, 128)
I:/ykato_git/datasets/oomugi_blender/test/o/0009.png <c

I:/ykato_git/datasets/oomugi_blender/test/o/0079.png <class 'numpy.ndarray'>
(1, 2, 128, 128)
(128, 128)
I:/ykato_git/datasets/oomugi_blender/test/o/0080.png <class 'numpy.ndarray'>
(1, 2, 128, 128)
(128, 128)
I:/ykato_git/datasets/oomugi_blender/test/o/0081.png <class 'numpy.ndarray'>
(1, 2, 128, 128)
(128, 128)
I:/ykato_git/datasets/oomugi_blender/test/o/0082.png <class 'numpy.ndarray'>
(1, 2, 128, 128)
(128, 128)
I:/ykato_git/datasets/oomugi_blender/test/o/0083.png <class 'numpy.ndarray'>
(1, 2, 128, 128)
(128, 128)
I:/ykato_git/datasets/oomugi_blender/test/o/0084.png <class 'numpy.ndarray'>
(1, 2, 128, 128)
(128, 128)
I:/ykato_git/datasets/oomugi_blender/test/o/0085.png <class 'numpy.ndarray'>
(1, 2, 128, 128)
(128, 128)
I:/ykato_git/datasets/oomugi_blender/test/o/0086.png <class 'numpy.ndarray'>
(1, 2, 128, 128)
(128, 128)
I:/ykato_git/datasets/oomugi_blender/test/o/0087.png <class 'numpy.ndarray'>
(1, 2, 128, 128)
(128, 128)
I:/ykato_git/datasets/oomugi_blender/test/o/0088.png <c